In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano' 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

Using Theano backend.


In [2]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = re.sub(r"\s{2,}", "", string)
    string = re.sub(r"[0-9]+", "", string)



    string = re.sub("#", " ", string)
    string = re.sub("!", " ", string)
    string = re.sub("@", " ", string)
    
    
    return string.strip().lower()

In [3]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

filter_sizes = [3, 5, 7]


# Lecture de DATASET Twitter.txt

In [4]:
# reading data

df=pd.read_csv("Tweets.txt",delimiter="\t")

df = df.dropna()
df = df.reset_index(drop=True)


df.columns = ['message', 'class']
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['class'])))


Shape of dataset  (9693, 2)
Index(['message', 'class'], dtype='object')
No. of unique classes 4


In [5]:
df

,message,class
0,أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر...,POS
1,البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام ال...,NEG
2,#الحرية_والعدالة | شاهد الآن: #ليلة_الاتحادية ...,OBJ
3,الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقول...,NEUTRAL
4,#انتخبوا_العرص #انتخبوا_البرص #مرسى_رئيسى #اين...,NEUTRAL
5,امير عيد هو اللي فعلا يتقال عليه ستريكر صريح #...,POS
6,أحكي لكم قصة حدثت بين شاب و أحد الفتيات كان بي...,OBJ
7,أدعوكم لحضور الندوة الثقافية الأربعاء مركز اعد...,OBJ
8,عودة جماعة الإخوان إلى الحياة السياسية بنفس وض...,NEG
9,_ علاج السكري #اف_نتشورال #السعودية #العراق #ا...,OBJ


In [6]:
macronum=sorted(set(df['class']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['class']=df['class'].apply(fun)

In [7]:
texts = []
labels = []


for idx in range(df.message.shape[0]):
    text = df.message[idx]
    texts.append(clean_str(str(text)))

for idx in df['class']:
    labels.append(idx)

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Number of Unique Tokens',len(word_index))

Number of Unique Tokens 37813


In [19]:
## Dataset after cleaning
texts

['أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر، بمناسبة صدور أولى روايته',
 'البرادعي يستقوى بامريكا مرةاخرى و يرسل عصام العريان الي واشنطن شئ مقرف',
 'الحرية_والعدالة | شاهد الآن:  ليلة_الاتحادية أول فيلم استقصائي يتناول أسرار و كواليس تعرض لأول مرة حول حقيقة',
 'الوالدة لو اقولها بخاطري حشيشة تضحك بس من اقولها ملل الله وكيلك تعطيني محاضرة عن الفسق والفجور بجنوب الشيشان  ليه كذا يانبع الحنان',
 'انتخبوا_العرص  انتخبوا_البرص  مرسى_رئيسى  اين_رئيسى  _  __ö',
 'امير عيد هو اللي فعلا يتقال عليه ستريكر صريح  كاريوكي  السكة_شمال',
 'أحكي لكم قصة حدثت بين شاب و أحد الفتيات كان بينهم حب كبير ولكن حدثت غلطة واحدة؟ فهل ستستمر هذه القصة ويتم',
 'أدعوكم لحضور الندوة الثقافية الأربعاء مركز اعداد القادة التفاصيل',
 'عودة جماعة الإخوان إلى الحياة السياسية بنفس وضعها السابق مستحيلة والطرمخة على جرائم الماضي لن تجعلنا نتقدم شبرا',
 '_ علاج السكري  اف_نتشورال  السعودية  العراق  اليمن  مصر  قطر  الكويت  عمان  سوريا  ليبيا',
 'الوطن | موجتان متعاقبتان لمقاومة صناعة الاستبداد (-) | عمرو حمزاوي:',
 'شاهد

In [9]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', data.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of Data Tensor: (9693, 1000)
Shape of Label Tensor: (9693, 4)


In [10]:
embeddings_index = {}
f = open('./glove.6B/glove.6B.100d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


# Etape 2 : representation en Vecteurs

KeyError: 'حسبته'

In [11]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [12]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(len(macronum), activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model.summary()
cp=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)

Simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         3781400   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 

In [13]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=2,callbacks=[cp])

Train on 7755 samples, validate on 1938 samples
Epoch 1/15
2076/7755 [=======>......................] - ETA: 3:07 - loss: 1.0860 - acc: 0.6657

KeyboardInterrupt: 

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves :CNN',fontsize=16)
fig1.savefig('loss_cnn.png')
plt.show()

In [ ]:
fig2=plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves : CNN',fontsize=16)
fig2.savefig('accuracy_cnn.png')
plt.show()